<a href="https://colab.research.google.com/github/pedrohenriquecordeiro/validation-of-machine-learning-models/blob/master/Valida%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Importa os dados
### Dados com registro de carros

In [8]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


### Separa os dados

In [0]:
import numpy as np
# fixa a aleatoriedade
SEED = 0
np.random.seed(SEED)

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score


# separa os dados - caracteristicas , marcacoes
x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

# separa os dados para treino e teste
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, 
                                     test_size = 0.25,
                                     stratify = y)
print(
    "Treinaremos com %d elementos e testaremos com %d elementos" % 
      (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


## Usa o dummyClassifier como baseline

In [11]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi de %.2f%%" % acuracia)

A acurácia do dummy stratified foi de 51.76%


## Classifica com uma arvore de decisão de profundidade 2

### SEED igual a 0

In [12]:
from sklearn.tree import DecisionTreeClassifier

modelo = DecisionTreeClassifier(max_depth = 2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print ("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 75.48%


Estamos vulneráveis a tomar uma decisão baseada em uma aleatoriedade. Não faz sentido validar, julgar se é bom ou ruim, de acordo com um número aleatório. Queremos minimizar o efeito dessa aleatoriedade, dessa escolha aleatória do treino, do teste e da DecisionTreeClassifier, na nossa decisão.

### com o SEED igual a 158020 a acuracia foi de 71.92%

 Em vez de apenas separarmos os dados em treino e teste (holdout), separaremos em N pedaços, para treinar, testar e validar o algoritmo de maneira cruzada. O nome deste processo é Cross Validation.

 A ideia é quebrarmos os dados em N pedaços e, então, rodarmos K processos de treino e teste, validando nosso algorítimo de maneira cruzada. Esse processo é chamado de K-fold, referente ao número (K) de vezes que quebraremos os dados para rodar a validação cruzada. 
 
 #### Nota K <= N.

# Usando o Cross Validate 
### O Cross Validate é uma validação! Não é um treinamento!

o cross validate é deterministico 

In [13]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

# 1- define 5 slitter classes
# 2- habilita a aleatoriedade no Cross Validade
#    separando os N grupos de forma aleatória
# 3- estratifica os dados de teste e treino
cv = StratifiedKFold( n_splits = 5 , shuffle = True)

modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo , x , y , 
                         cv = cv , 
                         return_train_score = False)

media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy mean: %.2f%%" % (media * 100))
print("Accuracy [%.2f%%, %.2f%%]" % 
      ((media - 2 *desvio_padrao) * 100, 
       (media + 2 * desvio_padrao) * 100))

Accuracy mean: 75.78%
Accuracy [74.25%, 77.31%]


## Criando o campo modelo para a VALIDAÇÃO ser mais realista
### Para efetuarmos a VALIDAÇÃO a partir do tipo de modelo de carro
#### Iremos adicionar ao dataset um campo que indica o modelo do carro em questão

In [14]:
import pandas as pd
carros = pd.read_csv('modelos-carro.csv')
carros.head()

,ID;IDMARCA;NOME
0,1;25;INTEGRA
1,2;25;LEGEND
2,3;25;NSX
3,4;27;MARRUA
4,5;28;145


#### Filtra os dados, selecionando apenas os nomes

In [15]:
names = []
import re

re1 = '.*?'	# Non-greedy match on filler
re2 = '((?:[a-z][a-z]+))'	# Word 1
rg = re.compile(re1+re2,re.IGNORECASE|re.DOTALL)

for index, row in carros.iterrows():
    text = row['ID;IDMARCA;NOME']
    m = rg.search(text)
    if m:
      name = m.group(1)
      names.append(name)

from random import shuffle
shuffle(names)

# print(names)
print(len(names))

1026


### Cria nova feature ao dataset dados
Essa feature será o nome do modelo.
O nome do modelo é estócastico, porém é baseado no ano do modelo

In [16]:
import numpy as np

# usamos uma faixa de valores para os modelos serem mais organicos dentro do dataset
num_sorteados = np.clip(
    dados.idade_do_modelo + np.random.randint(-10 ,10 + 1 , size = len(dados)),
    0,
    len(names))

# cria lista com os carros sorteados
carros_sorteados = [names[i] for i in num_sorteados]

# cria a nova coluna
dados['modelo'] = carros_sorteados
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,AXE
1,40557.96,1,20,12622.05362,PARK
2,89627.50,0,12,11440.79806,ANKONIAN
3,95276.14,0,3,43167.32682,RANGER
4,117384.68,1,4,12770.11290,HSC


# Separa treino e teste na validação cruzada, de acordo com os grupos
No mundo real, a qualquer momento pode surgir um novo modelo de carro e nosso algorítimo terá que saber lidar com esse modelo
#### Dessa forma, o teste vai ficar com apenas determinados tipo de grupos de modelos e o treino com outros diferentes grupos de modelos

É o caminho inverso de estratificar os dados

In [17]:
from sklearn.model_selection import GroupKFold

cv = GroupKFold(n_splits = 10)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, 
                         x,y, 
                         cv = cv, 
                         groups = dados.modelo, 
                         return_train_score=False)

media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy mean: %.2f%%" % (media * 100))
print("Accuracy [%.2f%%, %.2f%%]" % 
      ((media - 2 *desvio_padrao) * 100, 
       (media + 2 * desvio_padrao) * 100))

Accuracy mean: 75.78%
Accuracy [73.32%, 78.24%]


# Cria um pipeline

Teremos um processo de duas fases:

* Primeiro, o pré processamento que escala;
* depois, a validação.

Esse processo deve ser rodado várias vezes, de acordo com os nossos grupos e com os nossos splits. 

Para cada um dos processos do fold, serão rodados tanto a transformacao, quanto o estimador. 

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])
pipeline

Pipeline(memory=None,
         steps=[('transformacao',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('estimador',
                 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3,
                     gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [19]:
cv = GroupKFold(n_splits = 10)
results = cross_validate(pipeline, 
                         x,y, 
                         cv = cv, 
                         groups = dados.modelo, 
                         return_train_score=False)

media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy mean: %.2f%%" % (media * 100))
print("Accuracy [%.2f%%, %.2f%%]" % 
      ((media - 2 *desvio_padrao) * 100, 
       (media + 2 * desvio_padrao) * 100))

Accuracy mean: 76.84%
Accuracy [74.66%, 79.02%]
